# ***Tareas por Cumplir***


- [X] ***Convertir cada mintérmino por su equivalente binario.***


- [X] ***Agrupar minterminos por cantidad de unos.***


- [X] ***Comparar minterminos grupo superior con grupo inferior.***


- [X] ***Entrada nueva tabla de emparejamientos, repetir el paso 2 y paso 3, hasta no poder emparejar más.***


- [x] ***Elaborar tabla de primos.***


- [x] ***Encontrar primos esenciales.***


- [x] ***Retirar los implicantes cubiertos.***


# Clase Mintermino

Permite la construccion de objetos llamados Minterminos, al construirlo de esta manera se pueden administrar las caracteristicas de estos por medio de sus atributos y las funciones que los modifican

La funcion mas importante en esta clase es la que permite agrupar minterminos, pues compara su mintermino base con el mintermino que uno pueda elegir, este verifica la diferencia de sus implicantes y valores binarios, si sus valores binarios solo difieren en 1 bit procede a realizar la agrupacion



In [63]:
#Clase: Mintermino; manejo de minterminos base
class Mintermino:
#Constructor inicial de objetos MinTerminos
#Entradas: Implicantes, ValorBinario
    def __init__(self,implicantes,valorBinario):
        self._implicantes = implicantes
        self._implicantes.sort()
        self._valorBinario = valorBinario
        self._seleccionado = False
    #Funcion __str__ permite imprimir minterminos(IMPLICANTES) = Binario
    def __str__(self): 
        implicantes = ", ".join([str(valorBinario) for valorBinario in self._implicantes])
        return f"minterminos({implicantes}) = {self._valorBinario}" 
    #Funcion __eq__ permite comparar objetos
    def __eq__(self,mintermino):
        if type(mintermino) != Mintermino:
            return False
        return (self._valorBinario == mintermino._valorBinario and self._implicantes == mintermino._implicantes)
    #Funcion get_implicantes() obtiene todos los implicantes
    def get_implicantes(self):
        return self._implicantes
    #Funcion get_valorBinario() obtiene todos los implicantes
    def get_valorBinario(self):
        return self._valorBinario
    #Funcion get_seleccionado(): obtiene estado del mintermino
    def get_seleccionado(self):
        return self._seleccionado
    #Funcion seleccion(): Cambia estado de aquel mintermino seleccionado en la comparacion
    def seleccion(self):
        self._seleccionado = True
    #Funcion agrupar(mintermino): agrupa minterminos especificos que difieren menos de 1 bit
    #Retorna Objeto mintermino combinado
    def agrupar(self,mintermino):
        #Registro de diferencias de 1 y resultado
        diferencia = 0
        resultado = ""
        #Verificar si esta agrupando los mismos valores        
        if self._valorBinario == mintermino._valorBinario or self._implicantes == mintermino._implicantes:
            return None
        #Revisar cada bit
        for actual in range(len(self._valorBinario)):
            #Revisar bit de diferencia
            if self._valorBinario[actual] != mintermino._valorBinario[actual]:
                diferencia += 1
                resultado += "x"
            else:
                resultado +=self._valorBinario[actual]
            #La diferencia no debe ser mayor a 1 para combinarse
            if diferencia>1:
                return None
        return Mintermino(self._implicantes + mintermino._implicantes, resultado)
    
  

# Clase QuineMcCluskey

La clase que administra el algoritmo principal, posee funciones de gran importancia.

La funcion para convertir un numero decimal a binario.

La funcion iniciar_agrupamiento que crea un grupo para mantener las agrupaciones hechas por los minterminos.

La funcion primos implicantes la cual de manera recursiva va iterando por cada una los de grupos comparando los terminos verificando que se pueda agrupar.


In [116]:
#Clase: QuineMcCluskey; maneja procesamiento del algoritmo
class QuineMcCluskey:
#Constructor inicial clase QuineMcCluskey
#Entradas: Cantidad de Literales, Suma de Mintermino
    def __init__(self, literales, sumMinterminos):
        self._sumMinterminos = sumMinterminos
        self._literales = literales
    #Funcion conversion_binario(digito): Convierte digito en numero binario
    #Entrada: Digito a convertir
    #Retorna: Numero Binario
    def conversion_binario(self, digito):
        digito = bin(digito)[2:] #Convierta a binario y obtiene solo los numeros
        return digito.rjust(len(self._literales),"0") #Ajuste a la cantidad de variables
    #Funcion iniciar_agrupamiento() Empieza a agrupar por bits dados
    #Retorna una Lista que contiene listas de los grupos hechos
    def iniciar_agrupamiento(self):
        grupos = []
        #Lista adyacente de grupos depende de cantidad de variables
        for conteo in range(len(self._literales) + 1):
            grupos.append([])
        #Pasa por la suma de minterminos contando los 1s y agrupando
        for digito in self._sumMinterminos:
            conteo = self.conversion_binario(digito).count("1")
            grupos[conteo].append(Mintermino([digito],self.conversion_binario(digito)))
        return grupos
    #Funcion obtener_primos_implicantes() Se compara de manera recursiva los terminos de un grupo con el grupo adyacente
    def obtener_primos_implicantes(self, grupos = None):
        if grupos == None:
            grupos = self.iniciar_agrupamiento()
        # Solo 1 grupo -> retornar todos sus minterminos
        if len(grupos) == 1:
            return grupos[0]
        #Comparar con todos los demas
        else:
            noSeleccionados = []
            comparaciones = range(len(grupos)-1)
            nuevoGrupo = [[] for c in comparaciones]
            for compara in comparaciones:
                grupo1 = grupos[compara]
                grupo2 = grupos[compara + 1]
                for termino1 in grupo1:
                    for termino2 in grupo2:
                        #Intentar agrupar
                        termino3 = termino2.agrupar(termino1)
                        #Si termino1 != Termino2 -> Termino3 = None -> Añadir termino 3 a nuevo grupo
#print(termino3)
                        if termino3 != None:
                            termino1.seleccion()
                            termino2.seleccion()
                            if termino3 not in nuevoGrupo[compara]: 
                                nuevoGrupo[compara].append(termino3)
            #Obtiene lista de los minterminos que aun no se usan
            for grupo in grupos:
                for termino in grupo:
                    if termino.get_seleccionado() == False and termino not in noSeleccionados:
                        noSeleccionados.append(termino)
            #Uso de parte recursiva
            for termino in self.obtener_primos_implicantes(nuevoGrupo):
                if termino.get_seleccionado() == False and termino not in noSeleccionados:
                    noSeleccionados.append(termino)
            return noSeleccionados
    #Funcion covertura_implicantes(implicantes,implicantesPrimos) permite crear listas de las combinaciones de todos los posibles implicantes con la menor cantidad de estos
    def covertura_implicantes(self, implicantes, implicantesPrimos):
            implicantesPrimos = list(implicantesPrimos)
            lista_covertura = []
            lista_minima =[]
            for p in range(1, 2**len(implicantesPrimos)):
                lista_actual = []
                binario = bin(p)[2:]
                binario = binario.rjust(len(implicantesPrimos),"0")
                for i in range(len(binario)):
                    if binario[i] == "1":
                        lista_actual.append(implicantesPrimos[i])
                lista_covertura.append(lista_actual)
            nueva_lista_covetura = []  
            for sublistas in lista_covertura:
                temporal = []
                for implicante in sublistas:
                    for valorBinario in implicante.get_implicantes():
                        if valorBinario not in temporal and implicante in implicantes:
                            temporal.append(implicante)
                temporal.sort()
                if temporal == implicantes:
                    if len(sublista) < len(minSet):
                        lista_minima = sublista
            return lista_minima
    #Funcion runQM inicia el desarrollo final del algoritmo
    def runQM(self):
            implicantesPrimos = self.obtener_primos_implicantes(self.iniciar_agrupamiento()) 
            implicantesPrimosEscenciales = []
            implicantesSeleccionados = [False]*len(self._sumMinterminos)
            for i in range(len(self._sumMinterminos)):
                valorBinario = self._sumMinterminos[i]
                contador = 0
                final = None
                for mintermino in implicantesPrimos:
                    if valorBinario in mintermino.get_implicantes():
                        contador += 1
                        final = mintermino
                if contador == 1 and final not in implicantesPrimosEscenciales:
                    for impl in final.get_implicantes():
                        implicantesSeleccionados[self._sumMinterminos.index(impl)] = True
                        implicantesPrimosEscenciales.append(final)
            if implicantesSeleccionados.count(False)==0:
                return implicantesPrimosEscenciales
            implicantesPrimos = [implicantesPrimos for implicantesPrimos in implicantesPrimos if implicantesPrimos not in implicantesPrimosEscenciales ]
            if len(implicantesPrimos) == 1:
                return implicantesPrimosEscenciales + implicantesPrimos
            return implicantesPrimosEscenciales + self.covertura_implicantes([self._sumMinterminos[index] for index in range(len(self._sumMinterminos)) if not implicantesSeleccionados[index]], implicantesPrimos)
   #Funcion Run() permite iniciar la solucion del algoritmo obteniendo sus primos implicantes pasandolo a modo lectura
    def run(self):
        primosImplicantes = self.runQM()
        if(len(primosImplicantes)) == 0:
            return "0"
        if len(primosImplicantes) == 1:
            if primosImplicantes[0].get_valorBinario().count("x") == len(self._literales):
                return 1
        resultado = ""
        for y in range(len(primosImplicantes)):
            implicante = primosImplicantes[y]
            for x in range(len(implicante.get_valorBinario())):
                if implicante.get_valorBinario()[x] == "0" :
                    resultado += " ~"
                if implicante.get_valorBinario()[x] != "x" :
                    resultado += self._literales[x]
                if implicante.get_valorBinario().count("x", x+1) < len(implicante.get_valorBinario())-x-1 and implicante.get_valorBinario()[x]!= "x":
                    resultado += " * " 
            if y < len(primosImplicantes) -1: 
                resultado += " + "
        return resultado


In [117]:
QuineMcCluskeyObject = QuineMcCluskey(["a","b","c","d"],[0, 4, 5, 7, 8, 11, 12, 15])
QuineMcCluskeyObject.run()


' ~c *  ~d +  ~c *  ~d +  ~c *  ~d +  ~c *  ~d + a * c * d + a * c * d'